# Implementation of ChatBot using NLP

In [1]:
pip install nltk scikit-learn streamlit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
intents = [
    {
        "tag":"greeting",
        "patterns": ["Hi", "Hello", "Hey", "Whats up", "How are you"],
        "responses": ["Hi there", "Hello", "Hey", "Nothing much", "I am fine", "Thank you"]
    },
    {
        "tag":"goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "name",
        "patterns": ["What is your name?", "Do you have a name?", "Who are you?"],
        "responses": ["I'm your friendly chatbot!", "You can call me JARVIS!", "I am an AI-powered assistant."]
    },
    {
        "tag": "time",
        "patterns": ["What time is it?", "Can you tell me the time?", "What's the current time?"],
        "responses": ["I'm sorry, but I can't fetch real-time data. You can check the time on your device.", "You can look at the clock for the current time!"]
    },
    {
        "tag": "date",
        "patterns": ["What is today's date?", "Can you tell me the date?", "What's the date today?"],
        "responses": ["I'm unable to fetch real-time data, but you can check your calendar!", "Look at your device for today's date!"]
    },
    {
        "tag": "joke",
        "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes?"],
        "responses": ["Why don't scientists trust atoms? Because they make up everything!", "Why did the scarecrow win an award? Because he was outstanding in his field!", "I told my wife she should embrace her mistakes. She gave me a hug!"]
    },
    {
        "tag": "quote",
        "patterns": ["Give me a quote", "Motivational quote", "Inspire me"],
        "responses": ["Success is not final, failure is not fatal: it is the courage to continue that counts. – Winston Churchill", "Believe you can and you're halfway there. – Theodore Roosevelt", "The only limit to our realization of tomorrow is our doubts of today. – Franklin D. Roosevelt"]
    },
    {
        "tag": "programming",
        "patterns": ["What is Python?", "Tell me about Java", "What is programming?", "How do I learn coding?"],
        "responses": ["Python is a high-level programming language known for its simplicity and readability.", "Java is a widely used object-oriented programming language.", "Programming is the process of writing instructions that a computer can understand and execute.", "To learn coding, start with basic concepts, practice problem-solving, and work on projects!"]
    },
    {
        "tag": "math",
        "patterns": ["What is 2+2?", "Solve this equation", "Basic math question"],
        "responses": ["I can help with basic math concepts, but for solving equations, a calculator might be better!", "2+2 is 4!", "Math is fun! What else would you like to solve?"]
    },
    {
        "tag": "news",
        "patterns": ["What's the latest news?", "Tell me today's news", "Give me some updates"],
        "responses": ["I can't fetch live news, but you can check a news website for the latest updates.", "Try checking Google News or your favorite news app for the latest updates."]
    },
    {
        "tag": "sports",
        "patterns": ["Tell me about football", "Who won the last cricket match?", "Give me sports updates"],
        "responses": ["Football is a popular sport played worldwide, consisting of two teams trying to score goals.", "I'm unable to fetch live sports updates, but you can check ESPN or other sports news sources!", "Cricket is a bat-and-ball game played between two teams of eleven players."]
    },
    {
        "tag": "travel",
        "patterns": ["Where should I travel?", "Best travel destinations?", "Tell me about traveling"],
        "responses": ["Traveling is a great way to explore new cultures! Some popular destinations include Paris, Tokyo, and Bali.", "It depends on your interests! Do you like beaches, mountains, or cityscapes?", "A good travel experience depends on planning, budgeting, and exploring new cultures!"]
    },
    {
        "tag": "food",
        "patterns": ["What should I eat?", "Suggest me some food", "Best food to try?"],
        "responses": ["It depends on what you like! Do you prefer spicy, sweet, or savory foods?", "You can try some delicious pizza, sushi, or a classic burger!", "How about some homemade food? Cooking is fun and healthy!"]
    },
    {
        "tag": "movies",
        "patterns": ["Suggest me a movie", "What are some good movies?", "Best movies to watch?"],
        "responses": ["You can watch classics like 'Inception', 'The Godfather', or 'Interstellar'!", "Are you into action, comedy, or drama? There are so many great movies to choose from!", "Check IMDb or Netflix for trending movies!"]
    },
    {
        "tag": "technology",
        "patterns": ["Tell me about AI", "What is machine learning?", "How does blockchain work?"],
        "responses": ["Artificial Intelligence (AI) is a branch of computer science focused on building smart machines.", "Machine Learning is a subset of AI that enables computers to learn from data without being explicitly programmed.", "Blockchain is a decentralized digital ledger used for secure and transparent transactions."]
    },
    {
        "tag": "health",
        "patterns": ["How to stay healthy?", "Tips for good health", "Best exercises?"],
        "responses": ["To stay healthy, eat a balanced diet, exercise regularly, and get enough sleep.", "Exercise like running, yoga, and strength training can help keep you fit.", "Good health is a combination of physical, mental, and emotional well-being."]
    },
    {
        "tag": "fun_fact",
        "patterns": ["Tell me a fun fact", "Give me an interesting fact", "I want to learn something new"],
        "responses": ["Did you know honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3000 years old and still perfectly edible!", 
                      "Bananas are berries, but strawberries aren't!", 
                      "Octopuses have three hearts, and their blood is blue due to the presence of copper."]
    },
    {
        "tag": "exercise",
        "patterns": ["How can I stay fit?", "Suggest some exercises", "Workout tips?"],
        "responses": ["A good mix of cardio and strength training works wonders!", 
                      "You can do bodyweight exercises like push-ups, squats, and planks at home.", 
                      "Consistency is key! Even a 30-minute walk every day makes a big difference."]
    }
]